## Language Detection

Download the Dataset from https://www.kaggle.com/datasets/basilb2s/language-detection, then extract it and upload the csv file to this Notebook.

Credits: This Notebook is slightly adapted from https://github.com/basil-b2s/Language-Detector


In [3]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

import warnings
warnings.simplefilter("ignore")

Matplotlib is building the font cache; this may take a moment.


In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("basilb2s/language-detection")

print("Path to dataset files:", path)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 542k/542k [00:00<00:00, 10.7MB/s]

Extracting files...
Path to dataset files: /Users/siddartha/.cache/kagglehub/datasets/basilb2s/language-detection/versions/1


In [7]:
# Loading the dataset
data = pd.read_csv("Language Detection.csv")

In [8]:
data.head()

,Text,Language
0,"Nature, in the broadest sense, is the natural...",English
1,"""Nature"" can refer to the phenomena of the phy...",English
2,"The study of nature is a large, if not the onl...",English
3,"Although humans are part of nature, human acti...",English
4,[1] The word nature is borrowed from the Old F...,English


In [15]:
data.sample(10)

,Text,Language
9218,يمكن أن أقول ذلك دائمًا لأنني لم لقد استعدت جي...,Arabic
947,The original goal of the ANN approach was to s...,English
5369,Estaré contigo en un minuto.,Spanish
4792,"Ze begon al snel te huilen, ik wil dit niet, m...",Dutch
1058,now we often use the word awesome here's anoth...,English
9778,"Ich fürchte, ich habe keine Ahnung.",German
5004,"Actualmente, 67 son administradores humanos y ...",Spanish
4472,in Frankrijk werd eerdere afspraak door het le...,Dutch
6530,"извините, я не уловил этого.",Russian
527,The results were tabulated for several languag...,English


In [16]:
X = data["Text"]
y = data["Language"]

In [18]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [24]:
y

array([3, 3, 3, ..., 9, 9, 9], shape=(10337,))

In [25]:
le.classes_

array(['Arabic', 'Danish', 'Dutch', 'English', 'French', 'German',
       'Greek', 'Hindi', 'Italian', 'Kannada', 'Malayalam', 'Portugeese',
       'Russian', 'Spanish', 'Sweedish', 'Tamil', 'Turkish'], dtype=object)

In [26]:
data_list = []
for text in X:
    text = re.sub(r'[!@#$(),\n"%^*?\:;~`0-9]', ' ', text)
    text = re.sub(r'[[]]', ' ', text)
    text = text.lower()
    data_list.append(text)

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [28]:
# creating bag of words using countvectorizer

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
cv.fit(X_train)

x_train = cv.transform(X_train).toarray()
x_test  = cv.transform(X_test).toarray()

In [38]:
x_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(8269, 34624))

In [29]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(x_train, y_train)

MultinomialNB()

In [31]:
y_pred = model.predict(x_test)

In [34]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

ac = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

In [40]:
print("Accuracy is :",ac)

Accuracy is : 0.9811411992263056


In [41]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([('vectorizer', cv), ('multinomialNB', model)])
pipe.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('multinomialNB', MultinomialNB())])

In [42]:
y_pred2 = pipe.predict(X_test)
ac2 = accuracy_score(y_test, y_pred2)
print("Accuracy is :",ac2)

Accuracy is : 0.9811411992263056


In [43]:
with open('trained_pipeline-0.1.0.pkl','wb') as f:
    pickle.dump(pipe, f)

In [ ]:
# If you need to zip and download a folder, you can do it with this command:

# !zip -r ./trained_pipeline-0.1.0.pkl.zip ./trained_pipeline-0.1.0.pkl

In [45]:
text = "Hello, how are you?"
text = "Ciao, come stai?"
#text = "Hi, how are you?"
y = pipe.predict([text])
le.classes_[y[0]], y

('Italian', array([8]))